In [1]:
import keras
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Embedding, LSTM,Dense

Using TensorFlow backend.


In [2]:
top_words = 5000
(X_train, y_train),(X_test, y_test) = imdb.load_data(num_words=top_words)

In [34]:
INDEX_FROM = 3

In [35]:
#reverse lookup
word_to_id = keras.datasets.imdb.get_word_index()

In [36]:
word_to_id = {k:(v+INDEX_FROM) for k,v in word_to_id.items()}
word_to_id["<PAD>"] = 0
word_to_id["<START>"] = 1
word_to_id["<UNK>"] = 2
id_to_word = {value:key for key,value in word_to_id.items()}
print(' '.join(id_to_word[id] for id in X_train[0] ))

<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly <UNK> was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little <UNK> that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big <UNK> for the whole film but these children are amazing and should be <UNK> for what they

In [6]:
#one hot encode your documents
from numpy import array
from keras.preprocessing.text import one_hot
docs = ['Gut gemacht',
        'Gute arbeit',
        'Super idee',
        'Perfekt erledigt',
        'exzellent',
        'naja',
        'Schwache arbeit.',
        'Nicht gut',
        'Miese arbeit.',
        'Hätte es besser machen können.']
# integer encode the documents
vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs)

[[6, 5], [39, 45], [12, 14], [5, 21], [20], [13], [17, 45], [27, 6], [32, 45], [17, 31, 43, 27, 47]]


In [7]:
# Truncate and pad the review sequences 
from keras.preprocessing import sequence 
max_review_length = 500 
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length) 
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length) 

In [8]:
# Build the model 
embedding_vector_length = 32 
model = Sequential() 
model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length)) 
model.add(LSTM(100))
# model.add(Flatten()) # If you don't want to use LSTM
model.add(Dense(1, activation='sigmoid')) 
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy']) 
print(model.summary()) 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
#Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=3, batch_size=64) 

/home/vincent/anaconda2/envs/cds/lib/python3.6/site-packages/ipykernel/__main__.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  from ipykernel import kernelapp as app


Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 294s 12ms/step - loss: 0.4294 - acc: 0.7978 - val_loss: 0.3191 - val_acc: 0.8684
Epoch 2/3
25000/25000 [==============================] - 285s 11ms/step - loss: 0.2928 - acc: 0.8831 - val_loss: 0.3253 - val_acc: 0.8679
Epoch 3/3
25000/25000 [==============================] - 278s 11ms/step - loss: 0.2501 - acc: 0.9039 - val_loss: 0.3359 - val_acc: 0.8564
